In [13]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

In [14]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [15]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [16]:
analize = Analizer()
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_5_9_3,0.928487,0.895933,0.952475,0.945109,8.624492,7.592819,8.682015,8.105385,"Hidden Size=[16, 15], regularizer=0.02, learni..."
1,model_5_6_0,0.895256,0.956251,0.875372,0.918316,12.632209,4.984436,14.619789,9.519229,"Hidden Size=[16, 15], regularizer=0.02, learni..."
9,model_5_9_2,0.888619,0.751568,0.958455,0.910827,13.432622,18.125862,7.589537,13.167606,"Hidden Size=[16, 15], regularizer=0.02, learni..."
10,model_5_5_12,0.888581,0.790517,0.453798,0.651509,13.437160,19.046213,44.171448,30.869919,"Hidden Size=[16, 15], regularizer=0.02, learni..."
11,model_5_4_5,0.885966,0.786184,0.772070,0.777062,13.752520,13.391683,30.742620,21.556824,"Hidden Size=[16, 15], regularizer=0.02, learni..."
13,model_5_4_6,0.883614,0.756745,0.767589,0.764025,14.036140,15.235469,31.346973,22.817381,"Hidden Size=[16, 15], regularizer=0.02, learni..."
14,model_5_4_4,0.883549,0.808538,0.765613,0.780489,14.044016,11.991605,31.613571,21.225456,"Hidden Size=[16, 15], regularizer=0.02, learni..."
15,model_5_4_7,0.882233,0.728479,0.763998,0.751975,14.202741,17.005810,31.831324,23.982521,"Hidden Size=[16, 15], regularizer=0.02, learni..."
16,model_5_4_9,0.881504,0.688823,0.762585,0.737449,14.290693,19.489569,32.021938,25.387144,"Hidden Size=[16, 15], regularizer=0.02, learni..."
17,model_5_4_8,0.881058,0.700097,0.762860,0.741496,14.344462,18.783426,31.984850,24.995857,"Hidden Size=[16, 15], regularizer=0.02, learni..."
